In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import pandas as pd
import numpy as np







In [ ]:
import pandas as pd
from underthesea import word_tokenize
import regex as re
import string

train = pd.read_csv('data/vlsp_sentiment_train.csv', delimiter='\t')


test = pd.read_csv('data/vlsp_sentiment_test.csv', delimiter='\t')


new_df = train[['Data', 'Class']]


new_df_test = test[['Data', 'Class']]


emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

def clean_text(text):
    text = text.lower()
    text = re.sub(emoji_pattern, " ", text)
    text = re.sub(r'([a-z]+?)\1+',r'\1', text)
    text = re.sub(r"(\w)\s*([" + string.punctuation + "])\s*(\w)", r"\1 \2 \3", text)
    text = re.sub(r"(\w)\s*([" + string.punctuation + "])", r"\1 \2", text)
    # text = re.sub(r"(\d)([^\d.])", r"\1 \2", text)
    # text = re.sub(r"([^\d.])(\d)", r"\1 \2", text)
    text = re.sub(f"([{string.punctuation}])([{string.punctuation}])+",r"\1", text)
    text = text.strip()
    while text.endswith(tuple(string.punctuation+string.whitespace)):
        text = text[:-1]
    while text.startswith(tuple(string.punctuation+string.whitespace)):
        text = text[1:]
    text = re.sub(r"\s+", " ", text)
    return text






def preprocess_vietnamese_text(text):
    # Tokenization using underthesea
    text = clean_text(text)
    text = text.lower()
    #text = corrector(text,max_length=128)[0]['generated_text']

    #text = remove_stopwords(text)
    #text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    # try:
    #      text = corrector(text, max_length=256)[0]['generated_text']
    # except:
    #      pass

    #text = text.lower()
    tokens = word_tokenize(text)

    # Join tokens back into a single string
    tokens = [token for token in tokens]

    preprocessed_text = " ".join(tokens)


    return preprocessed_text

# Apply text preprocessing to 'Data' column directly
new_df['Data'] = new_df['Data'].apply(preprocess_vietnamese_text)

new_df_test['Data'] = new_df_test['Data'].apply(preprocess_vietnamese_text)

# Display the updated DataFrame
print(new_df.head())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_val, y_train, y_val = train_test_split(new_df['Data'], new_df['Class'], test_size=0.2, random_state=42)
X_test, y_test = new_df_test['Data'], new_df_test['Class']
# Sử dụng TF-IDF để vector hóa văn bản
vectorizer = TfidfVectorizer(max_features=5000)  # Thay đổi max_features tùy thuộc vào kích thước dữ liệu
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Sử dụng mô hình Naive Bayes Classifier
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)


MultinomialNB()

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = clf.predict(X_val_tfidf)

# Đánh giá mô hình
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)


Accuracy: 0.6823529411764706
Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.73      0.66       335
           1       0.75      0.72      0.74       327
           2       0.71      0.60      0.65       358

    accuracy                           0.68      1020
   macro avg       0.69      0.68      0.68      1020
weighted avg       0.69      0.68      0.68      1020



In [ ]:
y_pred_test = clf.predict(X_test_tfidf)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred_test)
report = classification_report(y_test, y_pred_test)

print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)


Accuracy: 0.6752380952380952
Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.70      0.64       350
           1       0.73      0.69      0.71       350
           2       0.72      0.64      0.68       350

    accuracy                           0.68      1050
   macro avg       0.68      0.68      0.68      1050
weighted avg       0.68      0.68      0.68      1050

